# GraphRAG

## Overview

This notebook demonstrates a **production-grade GraphRAG pipeline** using the `Semantica` framework. We go beyond simple tutorials to show how to build, analyze, and query a rich knowledge base using advanced graph algorithms and hybrid retrieval strategies.

### 🎯 What You Will Learn

1.  **Orchestration**: Using `Semantica` core to manage the full data lifecycle.
2.  **Advanced Analysis**: Using `GraphAnalyzer`, `CentralityCalculator`, and `CommunityDetector` to understand data structure.
3.  **Ontology Generation**: Automatically deriving a schema (classes & properties) from unstructured text.
4.  **Hybrid Search**: Combining Vector Similarity with Graph Traversal for superior context.

---

In [ ]:
# 🛠️ Dependencies
!pip install -qU semantica networkx matplotlib plotly pandas

## 1. Professional Setup & Imports

We organize imports by their functional role in the Semantica ecosystem.

In [ ]:
import os
import json
import pandas as pd

# --- Core Framework ---
from semantica.core import Semantica, ConfigManager

# --- Knowledge Graph & Analytics ---
from semantica.kg import (
    GraphBuilder, 
    GraphAnalyzer, 
    CentralityCalculator, 
    CommunityDetector
)

# --- Ontology & Schema ---
from semantica.ontology import OntologyGenerator

# --- Visualization ---
from semantica.visualization import KGVisualizer

print("✅ Semantica Enterprise Modules Loaded.")

## 2. Realistic Data Simulation

Instead of "Hello World", let's use a dense technical text. We'll simulate ingesting a research paper abstract on **"Graph Neural Networks for Knowledge Graph Completion"**. This provides rich entities (Models, Metrics, Concepts) and relationships.

In [ ]:
research_paper_text = """
Title: Learning Attention-based Embeddings for Relation Prediction in Knowledge Graphs.
Abstract:
Knowledge Graphs (KGs) are collections of structured facts, often represented as triplets (head, relation, tail).
However, most real-world KGs like Freebase, DBPedia, and YAGO are incomplete.
Link Prediction is the task of inferring missing facts based on existing ones.
We propose a new model, Graph Attention Network for Knowledge Graphs (GAT-KG), which utilizes a multi-head attention mechanism.
Unlike TransE or DistMult which rely on translational assumptions, GAT-KG captures the structural information of the local neighborhood.
Our encoder aggregates messages from neighbors, assigning different attention weights to different relationships.
The decoder uses a standard ConvKB layer to score the plausibility of triplets.
We evaluate our method on standard benchmarks: FB15k-237 and WN18RR.
Results show that GAT-KG outperforms state-of-the-art baselines like RotatE and TuckER by 5% in Hits@10.
Ablation studies confirm that the attention mechanism is crucial for handling sparse graphs.
"""

data_dir = os.path.join(os.getcwd(), "data")
os.makedirs(data_dir, exist_ok=True)
source_path = os.path.join(data_dir, "gnn_research.txt")

with open(source_path, "w", encoding="utf-8") as f:
    f.write(research_paper_text)

print(f"📄 Research paper saved to: {source_path}")

## 3. Configuration Management

We configure the system for **precision**. 
*   `entity_resolution_strategy: fuzzy`: Handles minor typos (e.g., "Conv-KB" vs "ConvKB").
*   `extraction`: We specify a capable LLM model to ensure we catch complex technical terms.

In [ ]:
config_data = {
    "project_name": "ResearchGraph",
    "embedding": {
        "provider": "openai",
        "model": "text-embedding-3-small"
    },
    "knowledge_graph": {
        "backend": "networkx",
        "merge_entities": True,
        "entity_resolution_strategy": "fuzzy",
        "similarity_threshold": 0.85
    },
    "extraction": {
        "model": "gpt-4o",
        "temperature": 0.0
    }
}

config = ConfigManager().load_from_dict(config_data)
semantica = Semantica(config=config)
semantica.initialize()

## 4. Building the Knowledge Base

We execute the orchestrated pipeline. This single call replaces manual parsing, cleaning, chunking, NER, RE, and embedding generation.

In [ ]:
print("🚀 Building Knowledge Base from Research Paper...")

result = semantica.build_knowledge_base(
    sources=[source_path],
    embeddings=True,
    graph=True,
    normalize=True
)

kg = result["knowledge_graph"]
print(f"\n✅ Extraction Complete.")
print(f"Nodes: {kg.number_of_nodes()} | Edges: {kg.number_of_edges()}")

## 5. Advanced Graph Analytics

A knowledge graph is more than just storage; it's a structure we can analyze to find insights.

### A. Centrality (Finding Key Concepts)
Who are the "VIPs" in this paper? We use Degree Centrality to find the most connected entities.

In [ ]:
centrality_calc = CentralityCalculator()
centrality_scores = centrality_calc.calculate_degree_centrality(kg)

# Convert to DataFrame for nice display
df_centrality = pd.DataFrame(centrality_scores.get("rankings", []))
if not df_centrality.empty:
    print("🏆 Top 5 Most Central Concepts:")
    print(df_centrality.head(5)[["node", "score"]].to_string(index=False))

### B. Community Detection (Topic Modeling)
Can we group these entities into topics? We use the **Louvain Algorithm** to detect communities within the graph.

In [ ]:
detector = CommunityDetector()
communities_result = detector.detect_communities(kg, algorithm="louvain")
communities = communities_result.get("communities", [])

print(f"🔍 Detected {len(communities)} distinct communities (topics).")

for i, comm in enumerate(communities[:3]):  # Show first 3
    print(f"\nGroup {i+1}: {', '.join(list(comm)[:5])}...")

## 6. Ontology Generation

One of Semantica's most powerful features is **Reverse Engineering the Schema**. We can look at the graph we just built and ask: "What is the underlying data model?"

In [ ]:
ontology_gen = OntologyGenerator()
ontology = ontology_gen.generate_from_graph(kg)

print("🧬 Inferred Ontology Classes:")
if "classes" in ontology:
    for cls in ontology["classes"][:5]:
        print(f"  - {cls.get('name')}: {cls.get('description', 'No description inferred')}")
else:
    print("  (Ontology generation requires sufficient data volume for inference)")

## 7. Hybrid GraphRAG Inference

Finally, we use the system to answer a complex question. We simulate the logic of a Hybrid Search:
1.  **Vector**: Find the relevant text chunk.
2.  **Graph**: Find the *neighbors* of the key entities to understand the context (e.g., what benchmarks were used?)
3.  **Synthesis**: (Conceptually) combine these for the LLM.

In [ ]:
query = "What benchmarks were used to evaluate GAT-KG?"
print(f"❓ Query: '{query}'")

# 1. Vector Search
vector_store = semantica.vector_store
embedding_gen = semantica.embedding_generator
q_vec = embedding_gen.generate_embeddings(query, data_type="text")
vec_results = vector_store.search(q_vec, k=1)

print("\n🔹 [Vector Retrieval] Found Context:")
top_chunk = vec_results[0].get('metadata', {}).get('text', '') if vec_results else "No match"
print(f"  \"{top_chunk[:100]}...\"")

# 2. Graph Traversal
print("\n🔸 [Graph Retrieval] Exploring Neighborhoods:")
# Find 'GAT-KG' in the graph and see what it connects to
target_node = None
for node in kg.nodes():
    if "GAT-KG" in str(node):
        target_node = node
        break

if target_node:
    neighbors = list(kg[target_node])
    print(f"  Entity '{target_node}' is connected to:")
    for n in neighbors:
        relation = kg[target_node][n].get('type', 'related_to')
        print(f"   --[{relation}]--> {n}")
else:
    print("  (Entity 'GAT-KG' not explicitly found in graph nodes)")

## 8. Visualization

We visualize the final graph structure.

In [ ]:
viz = KGVisualizer()

import matplotlib.pyplot as plt
import networkx as nx

plt.figure(figsize=(14, 10))
pos = nx.spring_layout(kg, k=0.8, seed=42)

# Draw with more style
nx.draw_networkx_nodes(kg, pos, node_size=1500, node_color="#6a0dad", alpha=0.8) # Semantica Purple
nx.draw_networkx_edges(kg, pos, width=1.5, alpha=0.4, edge_color="gray", arrows=True)
nx.draw_networkx_labels(kg, pos, font_size=9, font_color="white", font_weight="bold")

edge_labels = nx.get_edge_attributes(kg, 'type')
nx.draw_networkx_edge_labels(kg, pos, edge_labels=edge_labels, font_size=7)

plt.title("Research Paper Knowledge Graph", fontsize=20)
plt.axis('off')
plt.show()